## Passo 1: Verificar Python

Primeiro, vamos confirmar que o Python está funcionando.

In [1]:
import sys

print("Versão do Python:")
print(sys.version)
print()
print("Localização do Python:")
print(sys.executable)

Versão do Python:
3.13.9 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 19:11:29) [Clang 20.1.8 ]

Localização do Python:
/opt/anaconda3/bin/python


### Verificação

Se você viu a versão do Python acima (algo como 3.11 ou superior), está tudo certo!

Se deu erro, você precisa instalar Python primeiro.

## Passo 2: Instalar Biblioteca Requests

A biblioteca `requests` é usada para fazer requisições HTTP (buscar dados da internet).

Vamos verificar se ela está instalada:

In [2]:
try:
    import requests
    print("requests já está instalado!")
    print(f"Versão: {requests.__version__}")
except ImportError:
    print("requests NÃO está instalado.")
    print("Vamos instalar agora...")

requests já está instalado!
Versão: 2.32.5


### Se requests não estiver instalado

Execute a célula abaixo para instalar:

In [3]:
# Descomente a linha abaixo se precisar instalar
# !pip install requests

### Agora vamos testar se funciona:

In [4]:
import requests

print("requests importado com sucesso!")
print(f"Versão: {requests.__version__}")

requests importado com sucesso!
Versão: 2.32.5


## Passo 3: Entender o que é uma API

### O que é uma API?

API significa Application Programming Interface (Interface de Programação de Aplicações).

Pense em uma API como um **garçom em um restaurante**:
- Você (o código) faz um pedido
- O garçom (API) leva o pedido para a cozinha
- A cozinha (servidor) prepara
- O garçom (API) traz a resposta

### The Movie Database (TMDb)

O TMDb é um banco de dados de filmes com uma API gratuita.
Podemos pedir informações sobre filmes e eles nos respondem com dados reais.

### Como funciona:

1. Fazemos uma requisição HTTP (como abrir um navegador e digitar uma URL)
2. Enviamos nossa API key (como mostrar uma credencial)
3. Recebemos os dados em formato JSON

### O que é JSON?

JSON é um formato de texto para trocar dados. Parece com um dicionário Python:

```json
{
  "nome": "Matrix",
  "ano": 1999,
  "nota": 8.7
}
```

## Passo 4: Obter API Key do TMDb

Para usar a API do TMDb, você precisa de uma chave (API key).

### Como obter:

1. Vá para https://www.themoviedb.org/
2. Crie uma conta (gratuita)
3. Vá em Configurações > API
4. Solicite uma API key (escolha "Developer")
5. Copie a API Key (v3 auth)

### Guardar a API key:

Vamos criar um arquivo de configuração.

In [5]:
import json
from pathlib import Path

# Verificar se config.json existe
config_path = Path("config.json")

if config_path.exists():
    print("config.json já existe!")
    with open(config_path) as f:
        config = json.load(f)
    print("Configuração carregada.")
else:
    print("config.json NÃO existe.")
    print("Vamos criar um...")

config.json já existe!
Configuração carregada.


### Criar config.json

Execute a célula abaixo, mas **substitua 'SUA_API_KEY_AQUI'** pela sua chave real:

In [6]:
# IMPORTANTE: Substitua 'SUA_API_KEY_AQUI' pela sua API key real

config = {
    "tmdb": {
        "api_key": "SUA_API_KEY_AQUI",
        "base_url": "https://api.themoviedb.org/3",
        "language": "pt-BR"
    }
}

# Salvar arquivo
with open("config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("config.json criado com sucesso!")

config.json criado com sucesso!


### Verificar se salvou corretamente:

In [7]:
# Carregar config
with open("config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

print("Configuração atual:")
print(f"  Base URL: {config['tmdb']['base_url']}")
print(f"  Idioma: {config['tmdb']['language']}")
print(f"  API Key: {config['tmdb']['api_key'][:10]}...")

Configuração atual:
  Base URL: https://api.themoviedb.org/3
  Idioma: pt-BR
  API Key: SUA_API_KE...


## Passo 5: Fazer Primeira Requisição

Agora vamos buscar dados reais!

Vamos pedir informações sobre o filme "Clube da Luta" (ID: 550).

In [8]:
# Carregar configuração
with open("config.json") as f:
    config = json.load(f)

# Preparar URL e parâmetros
movie_id = 550  # Clube da Luta
base_url = config['tmdb']['base_url']
api_key = config['tmdb']['api_key']
language = config['tmdb']['language']

url = f"{base_url}/movie/{movie_id}"

print("Vamos fazer uma requisição para:")
print(url)
print()
print("Fazendo requisição...")

Vamos fazer uma requisição para:
https://api.themoviedb.org/3/movie/550

Fazendo requisição...


In [9]:
# Fazer requisição
response = requests.get(
    url,
    params={
        'api_key': api_key,
        'language': language
    }
)

print(f"Status da resposta: {response.status_code}")

Status da resposta: 401


### Entender Status Code

Status codes indicam o resultado da requisição:
- 200: Sucesso!
- 401: API key inválida
- 404: Não encontrado
- 500: Erro no servidor

Se você viu **200**, funcionou!

## Passo 6: Ver os Dados

Vamos ver o que a API nos enviou:

In [10]:
if response.status_code == 200:
    # Converter resposta para JSON (dicionário Python)
    filme = response.json()
    
    print("Dados do filme:")
    print()
    print(f"Título: {filme['title']}")
    print(f"Título original: {filme['original_title']}")
    print(f"Lançamento: {filme['release_date']}")
    print(f"Nota: {filme['vote_average']}/10")
    print(f"Número de votos: {filme['vote_count']}")
    print()
    print("Sinopse:")
    print(filme['overview'])
else:
    print(f"Erro: {response.status_code}")
    print(response.text)

Erro: 401
{"status_code":7,"status_message":"Invalid API key: You must be granted a valid key.","success":false}



### Ver estrutura completa do JSON

O JSON tem muito mais informações. Vamos ver tudo:

In [11]:
if response.status_code == 200:
    # Mostrar JSON formatado
    print(json.dumps(filme, indent=2, ensure_ascii=False))

### Explorar campos específicos

Vamos ver que tipos de dados estão disponíveis:

In [12]:
if response.status_code == 200:
    print("Campos disponíveis:")
    print()
    for campo in sorted(filme.keys()):
        valor = filme[campo]
        tipo = type(valor).__name__
        print(f"  {campo:20} ({tipo})")

## Passo 7: Salvar Dados Localmente

Vamos salvar estes dados em um arquivo para usar depois.

In [13]:
# Criar pasta para dados
dados_path = Path("dados")
dados_path.mkdir(exist_ok=True)

print(f"Pasta criada: {dados_path}")

Pasta criada: dados


In [14]:
if response.status_code == 200:
    # Salvar filme
    filme_path = dados_path / "filme_550.json"
    
    with open(filme_path, "w", encoding="utf-8") as f:
        json.dump(filme, f, indent=2, ensure_ascii=False)
    
    print(f"Filme salvo em: {filme_path}")
    print(f"Tamanho do arquivo: {filme_path.stat().st_size} bytes")

### Verificar que salvou corretamente:

In [15]:
# Ler arquivo salvo
with open("dados/filme_550.json", "r", encoding="utf-8") as f:
    filme_carregado = json.load(f)

print("Arquivo carregado com sucesso!")
print(f"Título: {filme_carregado['title']}")

Arquivo carregado com sucesso!
Título: Clube da Luta


## Resumo do Bloco 1

### O que fizemos:

1. Verificamos que Python está funcionando
2. Instalamos a biblioteca requests
3. Criamos arquivo de configuração (config.json)
4. Fizemos nossa primeira requisição HTTP
5. Recebemos dados reais em formato JSON
6. Salvamos os dados localmente

### O que aprendemos:

- O que é uma API
- Como fazer requisições HTTP
- O que é JSON
- Como trabalhar com arquivos em Python
- Como estruturar um projeto

### Arquivos criados:

```
config.json          - Configuração da API
dados/
  filme_550.json     - Dados do Clube da Luta
```

## Verificação Final

Execute esta célula para confirmar que tudo funcionou:

In [16]:
print("Verificação Final")
print("=" * 50)

# Verificar Python
print(f"Python: {sys.version_info.major}.{sys.version_info.minor}")

# Verificar requests
try:
    import requests
    print(f"requests: {requests.__version__}")
except:
    print("requests: NÃO instalado")

# Verificar config.json
if Path("config.json").exists():
    print("config.json: OK")
else:
    print("config.json: NÃO encontrado")

# Verificar pasta dados
if Path("dados").exists():
    arquivos = list(Path("dados").glob("*.json"))
    print(f"Pasta dados: OK ({len(arquivos)} arquivos)")
else:
    print("Pasta dados: NÃO encontrada")

print("=" * 50)
print()
print("Se tudo está OK, você está pronto para o próximo bloco!")

Verificação Final
Python: 3.13
requests: 2.32.5
config.json: OK
Pasta dados: OK (12 arquivos)

Se tudo está OK, você está pronto para o próximo bloco!


## Próximo Passo

Agora que temos acesso à API e sabemos buscar dados, vamos para o **Bloco 2**:

**02_coleta_dados.ipynb** - Vamos buscar vários filmes e começar a construir nosso pipeline de coleta.

### Pausa

Faça uma pausa de 5-10 minutos antes de continuar.
- Tome água
- Levante e estique
- Revise mentalmente o que aprendeu